# Importando as bibliotecas necessárias

In [1]:
from math import*
import numpy as np

# Declarando as variáveis necessárias

In [2]:
m = 3 #número de nós
n = 3 #número de elementos
E = 210e9 #módulo de elasticidade(Constante)
matriz_N = np.zeros([2,n]) #matriz dos nós
matriz_C = np.zeros([n,m]) #matriz conectividade
matriz_A = np.ones([m,1]) * 2e-4 #área da secção transversal (m²)
matriz_F = np.zeros([2*n,1]) 

# Adicionando a medida de cada nó

In [3]:
matriz_N[0,:] = [0,0,0.3]
matriz_N[1,:] = [0,0.4,0.4]

# Adicionando cada termo da conectividade 

### 0 = nó independente
### 1 = nó final
### -1 = nó inicial

In [4]:
matriz_C[0,0] = -1 
matriz_C[1,0] = 1

matriz_C[1,1] = -1
matriz_C[2,1] = 1

matriz_C[0,2] = -1
matriz_C[2,2] = 1

In [5]:
matriz_membros = matriz_N @ matriz_C #criação das matriz dos membros
print(matriz_membros)

[[0.  0.3 0.3]
 [0.4 0.  0.4]]


# Formação de matriz_S

In [6]:
lista_L = []
lista_matriz_S = []
    
for i in range(m):
    
    L = np.linalg.norm(matriz_membros[:,i])
    lista_L.append(L)
    c = matriz_membros[:,i]
    c.shape = [2,1]
    matriz_S = ((E * matriz_A[i,0])/L) * ((c) @ (c.T)) / (L**2)
    lista_matriz_S.append(matriz_S)
    
    print(matriz_S)

[[0.00e+00 0.00e+00]
 [0.00e+00 1.05e+08]]
[[1.4e+08 0.0e+00]
 [0.0e+00 0.0e+00]]
[[30240000.         40320000.        ]
 [40320000.         53760000.00000001]]


# Formação matriz de rigidez global

In [7]:
tamanho_S = len(matriz_S)
tamanho_C = len(matriz_C)
matriz_G = np.zeros([tamanho_S * tamanho_C,tamanho_S * tamanho_C])

for j in range(m):

    t = matriz_C[:,j]
    t.shape = [3,1]
    
    K = np.kron(((t) @ (t.T)), lista_matriz_S[j])
    matriz_G += K
    
print(matriz_G)

determinante = np.linalg.det(matriz_G)

[[ 3.0240e+07  4.0320e+07  0.0000e+00  0.0000e+00 -3.0240e+07 -4.0320e+07]
 [ 4.0320e+07  1.5876e+08  0.0000e+00 -1.0500e+08 -4.0320e+07 -5.3760e+07]
 [ 0.0000e+00  0.0000e+00  1.4000e+08  0.0000e+00 -1.4000e+08  0.0000e+00]
 [ 0.0000e+00 -1.0500e+08  0.0000e+00  1.0500e+08  0.0000e+00  0.0000e+00]
 [-3.0240e+07 -4.0320e+07 -1.4000e+08  0.0000e+00  1.7024e+08  4.0320e+07]
 [-4.0320e+07 -5.3760e+07  0.0000e+00  0.0000e+00  4.0320e+07  5.3760e+07]]


In [8]:
import numpy as np

# Definindo o vetor de constantes (F)
R1X = None
R1Y = 0

R2X = None
R2Y = None

R3X = 150
R3Y = -100

F = np.array([R1X, R1Y, R2X, R2Y, R3X, R3Y])
F.shape = (6, 1)
# Definindo as matrizes A e B de exemplo

# Encontrando índices onde A tem elementos None
linhas_a, colunas_a = np.where(F == None)

# Removendo as linhas e colunas correspondentes na matriz B
for linha, coluna in zip(linhas_a, colunas_a):
    matriz_G = np.delete(matriz_G, linha, axis=0)  # Removendo a linha
    matriz_G = np.delete(matriz_G, linha, axis=1)  # Removendo a coluna

print("\nMatriz G após a remoção das linhas e colunas correspondentes:")
print(matriz_G)



Matriz G após a remoção das linhas e colunas correspondentes:
[[ 1.5876e+08  0.0000e+00 -4.0320e+07]
 [ 0.0000e+00  1.4000e+08 -1.4000e+08]
 [-4.0320e+07 -1.4000e+08  1.7024e+08]]


# Declarar vetor global de forças concentradas

In [ ]:
# Resolvendo o sistema linear Ax = B
linhas_f, colunas_f = np.where(F == None)
for linha, coluna in zip(linhas_f, colunas_f):
    F[linha, coluna] = np.nan
    
u = np.linalg.solve(matriz_G, F)

print("\nA solução do sistema é:")
print(u)
